In [1]:
#%matplotlib widget
%pylab inline
# pylab.rcParams['figure.figsize'] = (8, 6)
import  general.utils as utils
import  datatool.testdata as testdata
import unified_ar.result_analyse.visualisation as vs
from sklearn.metrics import confusion_matrix
import  general.utils as utils
import unified_ar.result_analyse.resultloader
import unified_ar.result_analyse.visualisation as vs
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
from unified_ar.metric.CMbasedMetric import CMbasedMetric
from unified_ar.metric.event_confusion_matrix import event_confusion_matrix

import unified_ar.combiner.SimpleCombiner
comb=combiner.SimpleCombiner.EmptyCombiner()

def showres(dataset,real_events,pred_events,acts,weight,duration=None):
    real_events=vs.filterTime(real_events,duration)
    pred_events=vs.filterTime(pred_events,duration)
#     print(real_events)
#     print(pred_events)
#     utils.saveState((real_events,pred_events),'ali')
    event_cm     =event_confusion_matrix(real_events,pred_events,acts)
    quality      =CMbasedMetric(event_cm,'macro',None)
    quality_w    =CMbasedMetric(event_cm,'macro',weight)
    quality2     =CMbasedMetric(event_cm)
    print(f'quality={quality["f1"]} wq={quality_w["f1"]}')
    
#     print([f'{acts[i]}=>f={quality2["f1"][i]} p={quality2["precision"][i]} r={quality2["recall"][i]}' for i in range(len(acts))])
    
    # visualize(dataset)
    #vs.remove_gaps(real_events,pred_events)
    print('visualizing real and pred')
    vs.plotJoinAct(dataset,real_events,pred_events)
    from matplotlib.pylab import plt
    plt.show()
    

def groupize(datasetdscr,acts):
    #gacts=[[a] for a in datasetdscr.activities_map]
    #gacts.append([a for a in datasetdscr.activities_map])
    gacts=[[a] for a in acts[1:]]
    gacts.append([a for a in acts])
    return gacts

@interact
def result_selector(file=result_analyse.resultloader.get_runs()):
    if(file==None):return
    print('Analysing ',file)
#     info=utils.loadState(file,'info')
#     for i in info:
#         print(i,info[i])
    @interact_manual
    def continue_select():
        savedfile=file
        #result='A4Hr1'
        run_info,dataset,evalres=utils.loadState(savedfile)
        global a
        a=run_info,dataset,evalres
        print(run_info)
        #print(dataset)
        acts=[a for a in dataset.activities_map]
        gacts=groupize(dataset,acts)
        for fold in range(len(evalres)):
                global foldres
                foldres=evalres[fold]['test']
                print(f'fold----------{fold}')
         #       print(foldres)
                
                real_events=foldres.real_events
                pred_events=foldres.pred_events
                quality=foldres.quality
                s=real_events.iloc[0]['StartTime']
                e=s+pd.Timedelta('2 day')

    #             pred_events2=comb.combine2(foldres.times, foldres.predicted)
                showres(dataset,real_events,pred_events,acts,None,duration=[s,e])
    #             showres(dataset,real_events,pred_events2,acts,None,duration=[s,e])
                print('Evalution quality fold=%d is %s' % (fold, quality))


                for gindx,tacts in enumerate(gacts):
                #for gindx in foldres.functions:
                    weight=np.ones(len(acts))
                    for a in tacts:weight[a]=20
                    print('gindx %d - %s'% (gindx,foldres.functions[gindx]['segmentor']))

                    print(foldres.results[gindx].quality)
        #             print(foldres.results[gindx].event_cm)
        #             print(foldres.results[gindx].predicted_classes)
        #             print(foldres.results[gindx].Sdata.label)
                    if(gindx+1 in dataset.activities_map):
                        print(f'--------------{dataset.activities_map[gindx+1]}----------------')

                    real_events=foldres.results[gindx].real_events
                    pred_events=foldres.results[gindx].pred_events

                    times=[]
                    act_data=np.zeros((len(pred_events),12))
                    for i in range(len(pred_events)):
                        p=pred_events.iloc[i]
                        times.append({'begin':p['StartTime'],'end':p['EndTime']})
                        act_data[i,p['Activity']]=1
                    pred_events2=comb.combine2(times,act_data)
    #                 if hasattr(foldres.results[gindx],'Sdata'):
    #                     Sdata=foldres.results[gindx].Sdata
    #                     pred_events2=comb.combine(Sdata.s_event_list,Sdata.set_window, foldres.results[gindx].predicted)
                    showres(dataset,real_events,pred_events,acts,weight,duration=[s,e])            
                    showres(dataset,real_events,pred_events2,acts,weight,duration=[s,e])



        #             print(confusion_matrix(evalres[i].results[f].Sdata.label,evalres[i].results[f].predicted_classes))

            





Populating the interactive namespace from numpy and matplotlib


interactive(children=(Dropdown(description='file', options=(('210225_18-47-34-Home1', '210225_18-47-34-Home1')…

In [22]:

foldres.__dict__

{'name': 'Result',
 'shortrunname': 'SimplePreprocessing_Probabilistic_MaxActivityFetcher_KHistory_LSTMTest_EmptyCombiner_Accuracy_Accuracy_',
 'Sdata': <general.utils.Data at 0x7f284f3143c8>,
 'functions': {'preprocessor': ('SimplePreprocessing', {}),
  'segmentor': ('Probabilistic', {}),
  'activityFetcher': ('MaxActivityFetcher', {}),
  'featureExtractor': ('KHistory',
   {'k': 2, 'method': <feature_extraction.Simple.Simple at 0x7f284f314860>}),
  'classifier': ('LSTMTest', {'epochs': 10}),
  'combiner': ('EmptyCombiner', {}),
  'classifier_metric': ('Accuracy', {}),
  'event_metric': ('Accuracy', {})},
 'predicted': array([[5.1271945e-01, 1.0065294e-04, 1.4372031e-02, ..., 2.3738747e-02,
         1.8461103e-03, 2.6338008e-01],
        [5.1271945e-01, 1.0065294e-04, 1.4372031e-02, ..., 2.3738747e-02,
         1.8461103e-03, 2.6338008e-01],
        [5.1271945e-01, 1.0065294e-04, 1.4372031e-02, ..., 2.3738747e-02,
         1.8461103e-03, 2.6338008e-01],
        ...,
        [2.0722493

In [12]:
a[2][0].times[0]['begin']


Timestamp('2009-11-07 18:52:59.468001')

In [2]:
fold=2
print(f'fold{fold}')

fold2


In [3]:
dataset.activities_map

{0: 'null', 1: 'Act'}

In [1]:
import unified_ar.result_analyse.test_maker as tm

Utils loaded successfully!!
metric  ==>  ['classical', 'event_confusion_matrix2', 'CMbasedMetric', 'event_confusion_matrix', 'MyMetric', 'metric_abstract', 'EventBasedMetric']
